In [1]:
import pandas as pd
import sklearn
assert sklearn.__version__ >= "0.20"
import numpy as np
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

In [2]:
from sklearn.svm import SVC
from sklearn import datasets
from sklearn.datasets import fetch_california_housing

In [3]:
housing = fetch_california_housing()
X = housing["data"]
y = housing["target"]

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
"""
SVMs are sensitive to the feature scales. We need to scale the data (standardize features by 
removing the mean and scaling to unit variance) and can use standardscalar() to do this
"""
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [6]:
from sklearn.svm import LinearSVR

lin_svr = LinearSVR(random_state=42)
lin_svr.fit(X_train_scaled, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


LinearSVR(random_state=42)

In [7]:
from sklearn.metrics import mean_squared_error

y_pred = lin_svr.predict(X_train_scaled)
mse = mean_squared_error(y_train, y_pred)
mse

0.9641780189948642

In [8]:
#RMSE
np.sqrt(mse)

0.9819256687727764

In [9]:
"""
That is not very good. With this RMSE we can expect errors in the $10k range. Can we do better with an RBF (Radial Basis Function) Kernel? 
Lets use Randomized Search with Cross Validation to find appropriate hyperparameter values for C & Gamma
"""

from sklearn.svm import SVR
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import reciprocal, uniform

param_distributions = {"gamma": reciprocal(0.001, 0.1), "C": uniform(1, 10)}
rnd_search_cv = RandomizedSearchCV(SVR(), param_distributions, n_iter=10, verbose=2, cv=3, random_state=42)
rnd_search_cv.fit(X_train_scaled, y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] END .....C=4.745401188473625, gamma=0.07969454818643928; total time=  17.7s
[CV] END .....C=4.745401188473625, gamma=0.07969454818643928; total time=  10.1s
[CV] END .....C=4.745401188473625, gamma=0.07969454818643928; total time=  10.3s
[CV] END .....C=8.31993941811405, gamma=0.015751320499779724; total time=   9.9s
[CV] END .....C=8.31993941811405, gamma=0.015751320499779724; total time=   9.7s
[CV] END .....C=8.31993941811405, gamma=0.015751320499779724; total time=   9.8s
[CV] END ....C=2.560186404424365, gamma=0.002051110418843397; total time=   9.3s
[CV] END ....C=2.560186404424365, gamma=0.002051110418843397; total time=   9.4s
[CV] END ....C=2.560186404424365, gamma=0.002051110418843397; total time=  10.6s
[CV] END ....C=1.5808361216819946, gamma=0.05399484409787431; total time=   9.2s
[CV] END ....C=1.5808361216819946, gamma=0.05399484409787431; total time=   9.1s
[CV] END ....C=1.5808361216819946, gamma=0.05399

RandomizedSearchCV(cv=3, estimator=SVR(),
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f62a5af3e90>,
                                        'gamma': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f62a5a83410>},
                   random_state=42, verbose=2)

In [10]:
rnd_search_cv.best_estimator_

SVR(C=4.745401188473625, gamma=0.07969454818643928)

In [11]:
# Measure the RMSE on the training set

y_pred = rnd_search_cv.best_estimator_.predict(X_train_scaled)
mse = mean_squared_error(y_train, y_pred)
np.sqrt(mse)

0.5727524770785356

In [12]:
# Now evaluate on the test set

y_pred = rnd_search_cv.best_estimator_.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
np.sqrt(mse)

0.592916838552874